In [1]:
import sys
sys.path.append('..') # hack to allow us to import common helpers and still place this in a subdir
from common.helpers import load_config, load_contract, retry_call_wrapper

from web3 import Web3

In [2]:
config = load_config('../cfg.yaml')
web3 = Web3(Web3.HTTPProvider(config['rpc_endpoint']['http']['dwellir']))
cfg = config['venue']['stellaswap']
factory = load_contract(
    cfg['contract_address']['factory'],
    f"../{cfg['abi_path']['factory']}",
    web3,
)

In [3]:
def quote_with_fees(in_reserve, out_reserve, amount_in=1):
    amount_out = (amount_in * .9975 * out_reserve) / (in_reserve + amount_in * .9975)
    return amount_out

def inverse_quote_with_fees(in_reserve, out_reserve, amount_out):
    amount_in = (400/399) * amount_out * in_reserve / (out_reserve - amount_out)
    return amount_in

## Summary of sent orders

In [4]:
usdc_eth     = load_contract(factory.functions.getPair('0x818ec0A7Fe18Ff94269904fCED6AE3DaE6d6dC0b', '0xfA9343C3897324496A05fC75abeD6bAC29f8A40f').call(), f"../{cfg['abi_path']['pair']}", web3,)
eth_wglmr    = load_contract(factory.functions.getPair('0xfA9343C3897324496A05fC75abeD6bAC29f8A40f', '0xAcc15dC74880C9944775448304B263D191c6077F').call(), f"../{cfg['abi_path']['pair']}", web3,)
wglmr_stella = load_contract(factory.functions.getPair('0xAcc15dC74880C9944775448304B263D191c6077F', '0x0E358838ce72d5e61E0018a2ffaC4bEC5F4c88d2').call(), f"../{cfg['abi_path']['pair']}", web3,)
stella_usdc  = load_contract(factory.functions.getPair('0x0E358838ce72d5e61E0018a2ffaC4bEC5F4c88d2', '0x818ec0A7Fe18Ff94269904fCED6AE3DaE6d6dC0b').call(), f"../{cfg['abi_path']['pair']}", web3,)

In [5]:
eth_wglmr.functions.getReserves().call(block_identifier=1857467)

[37553713368671325325454, 12250567714118906135, 1663054590]

In [6]:
stella_usdc.functions.token0().call()

'0x0E358838ce72d5e61E0018a2ffaC4bEC5F4c88d2'

In [7]:
# r0, r1 ordering is manually done based on whether the LHS or RHS is token0 (see above function call)
b = 1857467
usdc_in = 34090718
r0, r1, _ = usdc_eth.functions.getReserves().call(block_identifier=b)
eth_out = quote_with_fees(r0, r1, usdc_in)
r1, r0, _ = eth_wglmr.functions.getReserves().call(block_identifier=b)
wglmr_out = quote_with_fees(r0, r1, eth_out)
r1, r0, _ = wglmr_stella.functions.getReserves().call(block_identifier=b)
stella_out = quote_with_fees(r0, r1, wglmr_out)
r0, r1, _ = stella_usdc.functions.getReserves().call(block_identifier=b)
usdc_out = quote_with_fees(r0, r1, stella_out)
# 160009.37114837766 is the expected_out_amount for this order, as noted in the logs (see 'Summary of sent orders')
usdc_out - usdc_in, usdc_out / usdc_in, usdc_in, usdc_out

(160009.37114837766, 1.0046936345297386, 34090718, 34250727.37114838)